In [16]:
import numpy as np
import tensorflow as tf
import pandas as pd # Untuk membuat DataFrame contoh

# Asumsikan rnn_layer.py berada di direktori yang sama atau dapat diimpor
from rnn_layer import Embedding_c, SimpleRNN_c, Bidirectional_c, Dense_c, Dropout_c, Model_c


def test_multi_layer_model_with_vectorized_input(
    batch_size=3, # Ubah batch_size agar sesuai dengan jumlah kalimat contoh
    output_sequence_length=7, # Panjang urutan setelah vektorisasi dan padding/truncating
    emb_dim=8,
    rnn_units=4,
    num_classes=3
):
    print(">>> Testing multi-layer model with TextVectorization: Embedding → Bidirectional(SimpleRNN) → Dense")

    # Set seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # ======= Data Contoh dan TextVectorization =======
    # Contoh data teks mentah
    sample_texts = [
        "ini adalah contoh kalimat pertama",
        "kalimat kedua untuk pengujian",
        "tes model rnn sederhana"
    ]
    if len(sample_texts) != batch_size:
        print(f"Warning: batch_size ({batch_size}) tidak sama dengan jumlah sample_texts ({len(sample_texts)}). Menyesuaikan batch_size.")
        batch_size = len(sample_texts)

    # Inisialisasi TextVectorization layer
    # max_tokens bisa disesuaikan, output_sequence_length menentukan 'timesteps'
    vectorize_layer = tf.keras.layers.TextVectorization(
        max_tokens=1000, # Ukuran vocabulary contoh
        output_mode='int',
        output_sequence_length=output_sequence_length
    )
    # Adapt vectorize_layer ke data teks contoh
    vectorize_layer.adapt(sample_texts)
    vocab_size_from_vectorizer = len(vectorize_layer.get_vocabulary())
    print(f"Ukuran Vocabulary dari TextVectorization: {vocab_size_from_vectorizer}")
    print(f"Vocabulary: {vectorize_layer.get_vocabulary()[:10]}...") # Tampilkan beberapa token

    # Ubah teks mentah menjadi urutan integer (input_data)
    # Ini akan menjadi input untuk Embedding layer di kedua model
    input_data = vectorize_layer(sample_texts)
    timesteps = output_sequence_length # timesteps sekarang ditentukan oleh output_sequence_length

    print(f"Input data (vectorized text) shape: {input_data.shape}") # (batch_size, timesteps)
    print(f"Contoh input_data (sampel pertama):\n{input_data[0].numpy()}")


    # ======= Build Keras Model =======
    # Input shape sekarang adalah (timesteps,) karena TextVectorization sudah diterapkan sebelumnya
    keras_input = tf.keras.Input(shape=(timesteps,), name="keras_input_tokens")
    embedding_layer_keras = tf.keras.layers.Embedding(
        input_dim=vocab_size_from_vectorizer, # Gunakan vocab size dari vectorizer
        output_dim=emb_dim,
        name="keras_embedding"
    )(keras_input)
    bidirectional_output_keras = tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=rnn_units, return_sequences=True, name="keras_simplernn_core"), # Beri nama pada RNN inti jika perlu
    merge_mode='concat',
    name="keras_bidirectional"
    )(embedding_layer_keras)

    dropout_output_keras = tf.keras.layers.Dropout(0.3, name="keras_dropout")(bidirectional_output_keras) # Tambahkan nama jika perlu

    dense_layer_keras = tf.keras.layers.Dense(
        units=num_classes,
        activation='softmax',
        name="keras_dense_output"
    )(dropout_output_keras) # Input dari dropout

    keras_model = tf.keras.Model(inputs=keras_input, outputs=dense_layer_keras)
    keras_model.summary(line_length=100) # Tampilkan summary model Keras

    # Dapatkan output dari Keras model
    # training=False penting untuk konsistensi jika ada layer seperti Dropout (meski tidak ada di sini)
    keras_output = keras_model(input_data, training=False).numpy()
    print(f"Keras output shape: {keras_output.shape}")


    # ======= Build Custom Model =======
    # Pastikan input_dim Embedding_c dan input_size untuk Dense_c sesuai
    # input_size untuk SimpleRNN_c adalah emb_dim
    # input_size untuk Dense_c setelah Bidirectional (concat) adalah rnn_units * 2
    custom_embedding = Embedding_c(input_dim=vocab_size_from_vectorizer, output_dim=emb_dim)
    custom_rnn = SimpleRNN_c(units=rnn_units, return_sequences=True, input_size=emb_dim)
    custom_bidirectional = Bidirectional_c(rnn_layer=custom_rnn, merge_mode='concat')
    # input_size untuk Dense_c setelah Bidirectional (concat) adalah rnn_units * 2
    # Jika SimpleRNN tidak return_sequences, maka input_size Dense_c adalah rnn_units*2 (jika BiRNN) atau rnn_units (jika SimpleRNN biasa)
    # Karena SimpleRNN_c di dalam Bidirectional_c di-set return_sequences=True,
    # dan Dense_c akan diterapkan ke setiap timestep, input_size-nya adalah output feature dari BiRNN.
    custom_dropout = Dropout_c(0.3)
    custom_dense = Dense_c(units=num_classes, activation_function='softmax', input_size=rnn_units * 2)


    custom_model = Model_c([
        custom_embedding,
        custom_bidirectional,
        custom_dropout,
        custom_dense
    ])

    # Load weights from Keras
    # Model_c.load_weights_from_keras_model akan mengambil bobot dari instance keras_model
    print("\nMemuat bobot dari model Keras ke model custom...")
    custom_model.load_weights_from_keras_model(keras_model)

    # Run custom model
    # training=False juga bisa ditambahkan jika Model_c Anda mendukungnya (misal untuk Dropout_c)
    custom_output = custom_model.predict(input_data, training=False) # Diasumsikan training=False adalah default atau diabaikan jika tidak ada Dropout
    print(f"Custom output shape: {custom_output.shape}")


    # ======= Compare Outputs =======
    # Output Keras dan Custom akan memiliki shape (batch_size, timesteps, num_classes)
    # karena SimpleRNN memiliki return_sequences=True dan Dense diterapkan per timestep.
    max_abs_diff = np.abs(keras_output - custom_output).max()
    mean_abs_diff = np.abs(keras_output - custom_output).mean()

    print(f"\nMax absolute difference: {max_abs_diff:.6g}")
    print(f"Mean absolute difference: {mean_abs_diff:.6g}")

    # Toleransi mungkin perlu disesuaikan tergantung presisi implementasi
    # Dengan seed yang sama dan transfer bobot, idealnya perbedaan sangat kecil.
    if np.allclose(keras_output, custom_output, atol=1e-5, rtol=1e-5):
        print("✅ Outputs match within tolerance (atol=1e-5, rtol=1e-5).")
        if batch_size > 0 and timesteps > 0:
            print("\nContoh output (sampel pertama, timestep pertama):")
            print(f"Keras : {keras_output[0, 0, :]}")
            print(f"Custom: {custom_output[0, 0, :]}")
    else:
        print("❌ Outputs do NOT match within the specified tolerance!")
        if batch_size > 0:
            print("\nKeras output (sampel pertama):\n", keras_output[0])
            print("\nCustom output (sampel pertama):\n", custom_output[0])
        else:
            print("\nKeras output:\n", keras_output)
            print("\nCustom output:\n", custom_output)
        print("\nCATATAN: Jika perbedaan sangat kecil (misalnya e-7 atau e-8), ini mungkin karena presisi floating point.")
        print("Jika perbedaan lebih besar, periksa kembali implementasi layer custom atau proses pemuatan bobot.")

    return max_abs_diff


# Run test
if __name__ == "__main__":
    test_multi_layer_model_with_vectorized_input()


>>> Testing multi-layer model with TextVectorization: Embedding → Bidirectional(SimpleRNN) → Dense
Ukuran Vocabulary dari TextVectorization: 14
Vocabulary: ['', '[UNK]', np.str_('kalimat'), np.str_('untuk'), np.str_('tes'), np.str_('sederhana'), np.str_('rnn'), np.str_('pertama'), np.str_('pengujian'), np.str_('model')]...
Input data (vectorized text) shape: (3, 7)
Contoh input_data (sampel pertama):
[11 13 12  2  7  0  0]


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_input_tokens (InputLayer)            │ (None, 7)                       │                 0 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ keras_embedding (Embedding)                │ (None, 7, 8)                    │               112 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ keras_bidirectional (Bidirectional)        │ (None, 7, 8)                    │               104 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ keras_dropout (Dropout)                    │ (None, 7, 8)                    │                 0 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ keras_dense_output (Dense)                 │ (None, 7, 3)                    │                27 │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 243 (972.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

Keras output shape: (3, 7, 3)

Memuat bobot dari model Keras ke model custom...
Custom output shape: (3, 7, 3)

Max absolute difference: 5.96046e-08
Mean absolute difference: 1.46646e-08
✅ Outputs match within tolerance (atol=1e-5, rtol=1e-5).

Contoh output (sampel pertama, timestep pertama):
Keras : [0.3607963  0.33049548 0.30870825]
Custom: [0.36079627 0.33049545 0.30870825]
